# **Lab Assignment # 02**

# **Applying Preprocessing on Sales missing dataset:**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#to ignore warnings:

import warnings
warnings.filterwarnings('ignore')

sd = pd.read_csv('sales_data_missing.csv')
sd.sample(5)

,store_id,customer_id,product_id,product_category,date,amount,single_price,transaction_id
74,Store 09,Customer 392,18582,Toys,9/5/2008 5:33,4.0,10.848014,75
17,Store 06,Customer 569,68351,Movies,12/27/2006 7:17,NaN,57.828898,18
96,Store 13,Customer 776,15381,Health,11/17/2006 0:55,2.0,81.335683,97
67,Store 12,Customer 539,32913,NaN,5/10/2005 1:43,8.0,99.156679,68
75,Store 14,Customer 1475,90692,Health,10/8/2006 6:29,5.0,56.732112,76


In [3]:
sd.set_index('transaction_id', inplace=True)
sd.head()

,store_id,customer_id,product_id,product_category,date,amount,single_price
transaction_id,,,,,,,
1,Store 01,Customer 1508,53642,Toys,4/1/2007 8:09,3.0,90.246230
2,Store 15,Customer 169,90945,Movies,2/15/2005 10:47,2.0,60.586312
3,Store 12,Customer 124,18548,Movies,9/27/2007 5:38,5.0,96.612695
4,Store 05,Customer 1988,85359,Books,5/7/2005 13:19,5.0,16.962671
5,Store 01,Customer 475,80069,Clothing,1/6/2008 17:36,5.0,65.215310


In [4]:
def calc_missing_values(df):
    missing_count = df.isnull().sum().sort_values(ascending=False)
    missing_percent = round(missing_count / len(df) * 100, 2)
    missing_values = pd.concat([missing_count, missing_percent], axis=1, keys=['Count', 'Percent'])
    return missing_values

print(calc_missing_values(sd))

                  Count  Percent
product_category      6      6.0
amount                6      6.0
store_id              0      0.0
product_id            0      0.0
customer_id           0      0.0
date                  0      0.0
single_price          0      0.0


In [5]:
sd['amount'].fillna(sd['amount'].mean(), inplace=True)

In [6]:
sd['product_category'].fillna(sd['product_category'].mode()[0], inplace=True)

In [8]:
from sklearn.preprocessing import LabelEncoder

store_encoder = LabelEncoder()
sd['store_id'] = store_encoder.fit_transform(sd['store_id'])

customer_encoder = LabelEncoder()
sd['customer_id'] = customer_encoder.fit_transform(sd['customer_id'])

In [9]:
sd.head()

,store_id,customer_id,product_id,product_category,date,amount,single_price
transaction_id,,,,,,,
1,0,25,53642,Toys,4/1/2007 8:09,3.0,90.246230
2,14,41,90945,Movies,2/15/2005 10:47,2.0,60.586312
3,11,8,18548,Movies,9/27/2007 5:38,5.0,96.612695
4,4,57,85359,Books,5/7/2005 13:19,5.0,16.962671
5,0,68,80069,Clothing,1/6/2008 17:36,5.0,65.215310


In [10]:
sd.drop(columns='date', axis=1, inplace=True)

In [11]:
sd = pd.get_dummies(sd, columns=['product_category'], drop_first=True)

In [13]:
first_quartile_amount = sd['amount'].quantile(0.25)
second_quartile_amount = sd['amount'].quantile(0.50)
third_quartile_amount = sd['amount'].quantile(0.75)

def amount_bucketizer(amount):
    if amount <= first_quartile_amount:
        return 0
    elif amount <= second_quartile_amount:
        return 1
    elif amount <= third_quartile_amount:
        return 2
    else:
        return 3

sd['amount'] = sd['amount'].apply(amount_bucketizer)

In [14]:
first_quartile_price = sd['single_price'].quantile(0.25)
second_quartile_price = sd['single_price'].quantile(0.50)
third_quartile_price = sd['single_price'].quantile(0.75)

def price_bucketizer(price):
    if price <= first_quartile_price:
        return 0
    elif price <= second_quartile_price:
        return 1
    elif price <= third_quartile_price:
        return 2
    else:
        return 3

sd['single_price'] = sd['single_price'].apply(price_bucketizer)

In [15]:
sd['product_category_Clothing'] = sd['product_category_Clothing'].apply(lambda x: 1 if x == True else 0)
sd['product_category_Electronics'] = sd['product_category_Electronics'].apply(lambda x: 1 if x == True else 0)
sd['product_category_Health'] = sd['product_category_Health'].apply(lambda x: 1 if x == True else 0)
sd['product_category_Home/Garden'] = sd['product_category_Home/Garden'].apply(lambda x: 1 if x == True else 0)
sd['product_category_Movies'] = sd['product_category_Movies'].apply(lambda x: 1 if x == True else 0)
sd['product_category_Sports'] = sd['product_category_Sports'].apply(lambda x: 1 if x == True else 0)
sd['product_category_Toys'] = sd['product_category_Toys'].apply(lambda x: 1 if x == True else 0)

In [16]:
sd.head()

,store_id,customer_id,product_id,amount,single_price,product_category_Clothing,product_category_Electronics,product_category_Health,product_category_Home/Garden,product_category_Movies,product_category_Sports,product_category_Toys
transaction_id,,,,,,,,,,,,
1,0,25,53642,0,3,0,0,0,0,0,0,1
2,14,41,90945,0,2,0,0,0,0,1,0,0
3,11,8,18548,2,3,0,0,0,0,1,0,0
4,4,57,85359,2,0,0,0,0,0,0,0,0
5,0,68,80069,2,2,1,0,0,0,0,0,0


In [17]:
sd.to_csv('sales_data_cleaned.csv')

In [18]:
scd = pd.read_csv('sales_data_cleaned.csv')
scd.sample(20)

,transaction_id,store_id,customer_id,product_id,amount,single_price,product_category_Clothing,product_category_Electronics,product_category_Health,product_category_Home/Garden,product_category_Movies,product_category_Sports,product_category_Toys
83,84,14,69,56873,0,1,0,0,0,0,0,0,0
2,3,11,8,18548,2,3,0,0,0,0,1,0,0
75,76,13,22,90692,2,2,0,0,1,0,0,0,0
38,39,12,2,28515,3,1,0,0,0,0,0,0,0
62,63,11,20,18863,2,0,0,0,0,0,1,0,0
76,77,14,75,97871,2,3,0,0,0,0,1,0,0
91,92,1,17,60439,0,2,0,0,0,0,0,1,0
99,100,8,91,14932,0,1,0,0,0,0,1,0,0
37,38,14,39,88467,2,1,0,1,0,0,0,0,0
81,82,13,14,85468,2,1,0,0,0,0,0,1,0


In [19]:
def calc_missing_values(df):
    missing_count = df.isnull().sum().sort_values(ascending=False)
    missing_percent = round(missing_count / len(df) * 100, 2)
    missing_values = pd.concat([missing_count, missing_percent], axis=1, keys=['Count', 'Percent'])
    return missing_values

print(calc_missing_values(scd))

                              Count  Percent
transaction_id                    0      0.0
store_id                          0      0.0
customer_id                       0      0.0
product_id                        0      0.0
amount                            0      0.0
single_price                      0      0.0
product_category_Clothing         0      0.0
product_category_Electronics      0      0.0
product_category_Health           0      0.0
product_category_Home/Garden      0      0.0
product_category_Movies           0      0.0
product_category_Sports           0      0.0
product_category_Toys             0      0.0
